# <center>Day-1 Practical Session, 25 May 2021</center>
## <center>Part 1: Epidemic Prevalence Estimation</center>
#### <center> *Li-Chun Zhang*<sup>1,2,3</sup> and *Melike Oguz-Alper*<sup>2</sup> </center>
  
##### <center> <sup>1</sup>*University of Southampton (L.Zhang@soton.ac.uk)*, <sup>2</sup>*Statistics Norway*, <sup>3</sup>*University of Oslo* </center>
***

### Illustration II: Estimation of change in prevalence over time under panel without tracing, panel ACS and iterative ACS designs

In this illustration, we will estimate the relative efficiencies of the change estimators under panel ACS and iterative ACS designs in comparison to those under panel design without adaptive tracing. The relative efficiency will be evaluated on the basis of the structure of the case-networks and the type of change in the structure over time.

***
We will run R-functions <font color=green>**gen.pop**</font> and <font color=green>**mainEpiPanel**</font> to generate the population and to calculate the relative efficiencies for different scenarios, respectively. 
***
<a id="gen-pop"></a>
#### Description of R-function <font color=green>**gen.pop**</font>
##### 1. Function parameters
* **N**: population size
* **theta**: a $2\times 1$ vector of prevalences at time points $t$ and $t+1$
* **M1**: number of case-networks at time $t$
* **M12**: number of new case-networks as such that **M12**$<$**M1**
* **Y12**: number of new cases. If $Y12>M1$, use a value divisible by **M1** to ensure $Y1=Y2$ when the prevalences same at both time points
* **emerge**: <font color=blue>**TRUE**</font> if *emerging* and <font color=blue>**FALSE**</font> if *evolving*

##### 2. Main steps of the function
* Equal case-network size at time point $t$, **K**, is calculated based on **N**, **theta[1]** and **M1**, and the total number of cases at $t$ is calculated by **Y1**=**M1*K**
* Equal case-network size at time point $t+1$ for new case-networks, **Kny**, is calculated based on **Y12** and **M12**, and the eventual number of cases at $t+1$ is calculated by **Y12**=**M12*Kny**
* $N\times 2$ array of index for case-networks, **kidx**, is created for all cases and set to $0$ for all non-cases
    * The first column of **kidx** refers to index for $t$: the first **Y1** units in the population take a network id number while the last $N-Y1$ units take value $0$, that is, $\underbrace{1,1,\dots,1}_{K \text{ times}},\underbrace{2,2,\dots,2}_{K \text{ times}},\dots,\underbrace{M1,M1,\dots,M1}_{K \text{ times}},\underbrace{0,0,\dots,0}_{(N-Y1) \text{ times}}$
    * The second column of **kidx** refers to index for $t+1$:
        * If **emerging**=<font color=blue>**TRUE**</font>: the first **Y1** units take the same ids as at $t$, the last **N-(Y1+Y12)** units in the population take value $0$, and the rest takes network id number existing only at $t+1$, that is,  $\underbrace{1,1,\dots,1}_{K \text{ times}},\underbrace{2,2,\dots,2}_{K \text{ times}},\dots,\underbrace{M1,M1,\dots,M1}_{K \text{ times}},\underbrace{M1+1,M1+1,\dots,M1+1}_{Kny \text{ times}},\underbrace{M1+2,M1+2,\dots,M1+2}_{Kny \text{ times}},\dots,\underbrace{M1+M12,M1+M12,\dots,M1+M12}_{Kny \text{ times}},\underbrace{0,0,\dots,0}_{(N-Y1-Y12) \text{ times}}$
        * If **emerging**=<font color=blue>**FALSE**</font>: the first **Y1** units take the same ids as at $t$, the last **N-(Y1+Y12)** units in the population take value $0$, and the rest takes network id number existing both at $t$ and $t+1$. For time point $t+1$, a random sample of size **M12** is selected without-replacement among the **M1** ids at time point $t$, that is,  $\underbrace{1,1,\dots,1}_{K \text{ times}},\underbrace{2,2,\dots,2}_{K \text{ times}},\dots,\underbrace{M1,M1,\dots,M1}_{K \text{ times}},\underbrace{i_1,i_1,\dots,i_1}_{Kny \text{ times}},\underbrace{i_2,i_2,\dots,i_2}_{Kny \text{ times}},\dots,\underbrace{i_{M12},i_{M12},\dots,i_{M12}}_{Kny \text{ times}},\underbrace{0,0,\dots,0}_{(N-Y1-Y12) \text{ times}}$, where the $i_m$, with $m\in1{\{1,2,\dots,M12\}}$, denote the sample ids that could take values in $\{1,2,\dots,M1\}$ 
* Finished number of cases from $t$ to $t+1$, **Y21**, is calculated by $Y21=N*theta[1]-(N*theta[2]-Y12)$
* Finished case-network size, **K21**, is obtained based on **Y21** and **M1**
* Identification of the ids for finished cases by either selecting the last **K21** units in each network at $t$ if $K21>0$ or selecting a random sample of size **Y21** without replacement from **Y1** units in the population, otherwise
* The **Y21** elements corresponding to the selected finished cases among the first **Y1** elements in the second column of the index array **kidx** is replaced with $0$ 
    * For example, in case $K21=2$, the first **Y1** indexes of **kidx[,2]** will be
    
    $\underbrace{1,1,\dots,1,0,0}_{K \text{ times}},\underbrace{2,2,\dots,2,0,0}_{K \text{ times}},\dots,\underbrace{M1,M1,\dots,M1,0,0}_{K \text{ times}}$

##### 3. Main outputs of the function
* A list of index of case- and non-case networks for time points $t$ and $t+1$


#### Description of R-function <font color=green>**mainEpiPanel**</font>
##### 1. Function parameters
* **N**: population size
* **theta**: a $2\times 1$ vector of prevalences at time points $t$ and $t+1$
* **M1**: number of case-networks at time $t$
* **M12**: number of new case-networks as such that **M12**$<$**M1**
* **Y12**: number of new cases. If $Y12>M1$, use a value divisible by **M1** to ensure $Y1=Y2$ when the prevalences same at both time points
* **emerge**: <font color=blue>**TRUE**</font> if *emerging* and <font color=blue>**FALSE**</font> if *evolving*
* **f**: sampling fraction
* **lift**:  the odds-ratio between cases and non-cases, denoted by $\eta$ (see [Illustration I: Size-biased sampling and adaptive network tracing](Day1_PS_epidemic.ipynb#size-biased))
* **B**: number of replications 
* **go**: use <font color=blue>**TRUE**</font> to get analytic standard error of the change estimator under panel ACS and the relative efficiency against the change estimator under panel design and use <font color=blue>**FALSE**</font> otherwise


##### 2. Main steps of the function
* R-function [gen.pop](#gen-pop) is complied to get indexes for case- and non-case networks at time points $t$ and $t+1$
* A $2\times 1$ vector of total cases is obtained based on **kidx**, denoted by **Y** inside the function
* An array of dimension $N\times 2$ of $y$ dummy variables being equal to $1$ for cases and $0$ for non-cases
* A $2\times 1$ vector of maximum case-network sizes at each time point, $K=(K1,K2)^\top$, is created, denoted by **K** inside the function
* Sample size $n_0$, **n0** inside the function, is calculated based on **N** and **f**
* Inclusion probabilities at $t$, $\pi_i=\mathrm{Pr}(i\in s_0)$, denoted by **p** inside the function, are calculated as proportional to **lift** if $y_i=1$, and $1$ otherwise
* Inclusion probabilites of case-networks, denoted by **pr.k** inside the function, are calculated by 
    * $\pi_{(\kappa)}=1-(1-p.1)^{m_\kappa}$, where p.1=max(p), under Poisson sampling of $s_0$, where $m_\kappa$ is the size of the case-network $\kappa$
* Two arrays, **n1idx** and **n2idx**, with dimensions $N\times 2$ are created. The first columns are replaced with number of cases in the corresponding networks. Non-cases have value $0$. The second columns are replaced with the inclusion probabilities of the corresponding networks. Non-cases have value $n_0/N$.
* Sampling variances of the HTE of change in prevalence from $t$ to $t+1$ under panel design are calculated by 
    * Under SRS of $s_0$: **v.srs**
$$V_{srs}(\hat{\Delta}_{t,t+1}^{panel})=\big(1-\frac{n_0}{N}\big)\frac{1}{N-1}\frac{\sum_{i\in N}(z_i-\Delta_{t,t+1}^{panel})^2}{n_0},\quad z_i=y_2-y_1, \quad \Delta_{t,t+1}^{panel}=\theta_2-\theta_1$$
    * Under poisson sampling of $s_0$: **v.pois**
    $$V_{pois}(\hat{\Delta}_{t,t+1}^{panel})=\frac{1}{N^2}\sum_{i\in N}\big(\frac{1}{\pi_i}-1\big)(y_2-y_1)^2$$
* Sampling variances of the HTE of prevalences at $t$ and $t+1$ under panel ACS design are calculated by implementing the formula (4.2) in the Lecture Notes. For simplicity in coding, the covariance term is ignored for variance at $t+1$ as the variance term dominates the variance. 
* If **go**=<font color=blue>**TRUE**</font>: The sampling variance of the HTE of the change in prevalence from $t$ to $t+1$ is calculated by
$$\mathrm{V}(\hat{\Delta}_{t,t+1}^{pACS})=\mathrm{V}(\hat{\theta}_{t})+\mathrm{V}(\hat{\theta}_{t+1})-2\mathrm{Cov}(\hat{\Delta}_{t,t+1}^{pACS}),$$
where the covariance term is obtained by implementing the covariance formula in Section 4.4.2 in the Lecture notes
* Simulation study
    * **B** random samples selected with *Sequential Poisson Sampling* (SPS) from the population for time point $t$ 
    * Sample units in $s(t)$ and $s(t+1)$ are obtained under panel, panel ACS and iterated ACS designs
    * For each pair of random samples at time points $t$ and $t+1$, the change of prevalence is estimated under (see formulas for the estimators in Section 4.4.2 in Lecture Notes)
        * panel design without tracing, based on $s(t)=s_0$ over time 
        * panel ACS: $s(t)$ based on $s_0$ and $A_{t}$;  $s(t+1)$ based on $s_0$ and $A_{t+1}$ 
        * iterated ACS design: $s(t)$ based on $s_0$ and $A_{t}$;  $s(t+1)$ based on $s(t)$ and $A_{t+1}$ 

##### 2. Main outputs of the function
* Monte-Carlo expectations of the change estimators under three designs
* Monte-Carlo standard errors of the change estimators under three designs
* Relative efficiencies of panel ACS and iterated ACS designs against panel design without adaptive tracing

***
##### Possible choices of function parameters (**M1**, **M12**, **Y12**, **emerge**) for populations with different dynamics over time
* L1, Large, Quickly Evolving: (10, 2, 200, F)
* L2, Large, Quickly Emerging: (10, 2, 200, T)
* L3, Large, Slowly Emerging: (10, 5, 100, T)

* M1, Medium, Quickly Evolving: (100, 10, 400, F)
* M2, Medium, Quickly Emerging: (100, 10, 400, T)
* M3, Medium, Slowly Emerging: (100, 10, 100, T)

* S1, Small, Quickly Evolving: (500, 10, 400, F)
* S2, Small, Quickly Emerging: (500, 10, 400, T)
* S3, Small, Slowly Emerging: (500, 50, 100, T)
***

In [10]:
gen.pop <- function(N=10^5,theta=c(0.01,0.01),M1=50,M12=10,Y12=100,emerge=T)
{
# at t
  K = trunc(N*theta[1]/M1 + 0.5); Y1 = M1*K
  kidx = array(0,c(N,2))
  kidx[1:Y1,1] = c(t(array(1:M1,c(M1,K))))
# at t+1  
#  Kny = trunc(min(K, Y12/M12) + 0.5); Y12 = M12*Kny
  Kny = trunc(Y12/M12+0.5); Y12 = M12*Kny
  if (emerge) { kidx[Y1+c(1:Y12),2] = c(t(array(M1+c(1:M12),c(M12,Kny)))) }
  else { kidx[Y1+c(1:Y12),2] = c(t(array(sample.int(M1,M12,replace=F),c(M12,Kny)))) }
  Y21 = N*theta[1] - (N*theta[2] - Y12)
  kidx[1:Y1,2] = kidx[1:Y1,1]
  K21 = trunc(Y21/M1)
  if (K21>0) { s = c(t(array(K*c(1:M1),c(M1,K21))) - c(1:K21) +1) }
  else { s = sample.int(Y1,Y21,replace=F) }
  kidx[s,2] = 0
  list(kidx=kidx)
}

In [11]:
mainEpiPanel <- function(N=10^5,theta=c(0.01,0.01),M1=50,M12=10,Y12=100,emerge=T,f=0.01,lift=1,B=50,go=F)
{
  kidx = gen.pop(N=N,theta=theta,M1=M1,M12=M12,Y12=Y12,emerge=emerge)$kidx
  Y = colSums(kidx>0)
  y = 1*(kidx>0) 
  K = c(max(table(kidx[y[,1]>0,1])), max(table(kidx[y[,2]>0,2])))
  cat("(M, max(K(t,t+1))) =",c(length(unique(c(kidx)))-1, K),"\t Y =",Y,"\n")

  n0 = trunc(N*f+0.5)
  p = rep(n0/N,N); p[1:Y[1]] = lift*p[1:Y[1]]; p = n0*p/sum(p); p.1 = max(p)
  pr.k = 1-exp(c(1:max(K))*log(1-p.1)); 
    cat('Summary of pr.k:','\n')
    print(summary(pr.k))
  n1idx = n2idx = t(array(c(min(p)),c(2,N)))
  for (k in 1:M1) { idk = kidx[,1]==k; n1idx[idk,1] = sum(idk); n1idx[idk,2] = pr.k[sum(idk)] }
  for (k in unique(kidx[,2])) { if (k>0) {
    idk = kidx[,2]==k; n2idx[idk,1] = sum(idk); n2idx[idk,2] = pr.k[sum(idk)] }}
  
  v.srs = (1-n0/N)*var(y[,2]-y[,1])/n0
  v.pois = sum((1/p-1)*(y[,2]-y[,1])^2)/N^2
  cat("SE(initial) by SRS, Pois =",sqrt(c(v.srs,v.pois)),"\n")
  tmp = 1 + exp(2*K[1]*log(1-p.1)) - 2*exp(K[1]*log(1-p.1))
  v1.acs = c(M1*(1/pr.k[K[1]]-1), M1*(M1-1)*(tmp/pr.k[K[1]]^2 -1))*K[1]^2/N^2
  cat("SE(ACS_1) =",sqrt(sum(v1.acs)),"\t leading term:",sqrt(v1.acs[1]),"\n")
  i2k = unique(kidx[kidx[,2]>0,2]); v2.acs = 0
  for (k in i2k) { m = sum(kidx[,2]==k); v2.acs = v2.acs + (1/pr.k[m]-1)*m^2/N^2 }
  cat("SE(ACS_2) =",sqrt(v2.acs),"\n")
  if (go) {  
    i1k = unique(kidx[kidx[,1]>0,1]); cov.acs = 0
    for (k1 in i1k) { for (k2 in i2k) {
      m1 = sum(kidx[,1]==k1); m2 = sum(kidx[,2]==k2); m12 = sum(kidx[,1]==k1 | kidx[,2]==k2)
      tmp = 1 - exp(m1*log(1-p.1)) - exp(m2*log(1-p.1)) + exp(m12*log(1-p.1))
      cov.acs = cov.acs + (tmp/(pr.k[m1]*pr.k[m2]) -1)*m1*m2/N^2 }}
    v.pacs = sum(v1.acs) + v2.acs - 2*cov.acs
    cat("SE(ACS_panel) =",sqrt(v.pacs),"\t RE(analytic) =",sum(v.pacs)/v.pois,"\n")
  }
  
  mat = array(0,c(B,3)); t1 = rep(0,B)
  p1 = p; p1[kidx[,1]>0] = n1idx[kidx[,1]>0,2]
  for (i in 1:B) {
# s0 by SPS (sequential Poisson sampling), s0y = effective sample of cases 
    u = runif(N,0,1)/p
    s0 = sort((c(1:N)[order(u)])[1:n0])
# HT estimator under panel design    
    mat[i,1] = sum((y[s0,2]-y[s0,1])/p[s0])/N
# HT estimator under panel ACS
    if (sum(y[s0,1])>0) { s0y = s0[y[s0,1]>0]
      idk = s0y[!duplicated(kidx[s0y,1])]
      t1[i] = sum(n1idx[idk,1]/n1idx[idk,2])/N }
    if (sum(y[s0,2])>0) { s0y = s0[y[s0,2]>0]
      idk = s0y[!duplicated(kidx[s0y,2])]
      mat[i,3] = sum(n2idx[idk,1]/n2idx[idk,2])/N }
    mat[i,2] = mat[i,3] - t1[i]
# Iterated ACS, based on samples s(t) and A(t+1)
    s.t = s0[y[s0,1]==0]
    if (sum(y[s0,1])>0) { s0y = s0[y[s0,1]>0]
      idk = unique(kidx[s0y,1])
      for (k in idk) { s.t = c(s.t, c(1:N)[kidx[,1]==k]) }} 
    mat[i,3] = sum(y[s.t,2]/p1[s.t])/N - t1[i]
  }
  
  emat = rbind(colMeans(mat), sqrt(diag(var(mat))))
  colnames(emat) = c("Panel","pACS","iACS")
  rownames(emat) = c("MC-Mean","MC-SD")
  print(emat)
  cat("RE(pACS, iACS) =",emat[2,2:3]^2/emat[2,1]^2,"\n")
}

In [12]:
table(gen.pop()$kidx[,1])


    0     1     2     3     4     5     6     7     8     9    10    11    12 
99000    20    20    20    20    20    20    20    20    20    20    20    20 
   13    14    15    16    17    18    19    20    21    22    23    24    25 
   20    20    20    20    20    20    20    20    20    20    20    20    20 
   26    27    28    29    30    31    32    33    34    35    36    37    38 
   20    20    20    20    20    20    20    20    20    20    20    20    20 
   39    40    41    42    43    44    45    46    47    48    49    50 
   20    20    20    20    20    20    20    20    20    20    20    20 

In [13]:
table(gen.pop()$kidx[,2])


    0     1     2     3     4     5     6     7     8     9    10    11    12 
99000    18    18    18    18    18    18    18    18    18    18    18    18 
   13    14    15    16    17    18    19    20    21    22    23    24    25 
   18    18    18    18    18    18    18    18    18    18    18    18    18 
   26    27    28    29    30    31    32    33    34    35    36    37    38 
   18    18    18    18    18    18    18    18    18    18    18    18    18 
   39    40    41    42    43    44    45    46    47    48    49    50    51 
   18    18    18    18    18    18    18    18    18    18    18    18    10 
   52    53    54    55    56    57    58    59    60 
   10    10    10    10    10    10    10    10    10 

In [14]:
table(gen.pop(Y12=63)$kidx[,2]) # When Y12>M1


    0     1     2     3     4     5     6     7     8     9    10    11    12 
98990    19    19    19    19    19    19    19    19    19    19    19    19 
   13    14    15    16    17    18    19    20    21    22    23    24    25 
   19    19    19    19    19    19    19    19    19    19    19    19    19 
   26    27    28    29    30    31    32    33    34    35    36    37    38 
   19    19    19    19    19    19    19    19    19    19    19    19    19 
   39    40    41    42    43    44    45    46    47    48    49    50    51 
   19    19    19    19    19    19    19    19    19    19    19    19     6 
   52    53    54    55    56    57    58    59    60 
    6     6     6     6     6     6     6     6     6 

In [15]:
mainEpiPanel(M1=10,M12=2,Y12=200,emerge=F,f=0.01,lift=1,B=50,go=T) # Large, quickly evolving

(M, max(K(t,t+1))) = 10 100 180 	 Y = 1000 1000 
Summary of pr.k: 
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0100  0.3686  0.5973  0.5401  0.7432  0.8362 
SE(initial) by SRS, Pois = 0.001989985 0.001989975 
SE(ACS_1) = 0.002402847 	 leading term: 0.002402847 
SE(ACS_2) = 0.002327396 
SE(ACS_panel) = 0.001671767 	 RE(analytic) = 0.7057591 
              Panel        pACS         iACS
MC-Mean 0.000700000 0.000287036 0.0003060874
MC-SD   0.001729103 0.001450867 0.0013293066
RE(pACS, iACS) = 0.7040669 0.591029 


In [16]:
mainEpiPanel(M1=10,M12=2,Y12=200,emerge=T,f=0.01,lift=1,B=50,go=T) # Large, quickly emerging

(M, max(K(t,t+1))) = 12 100 100 	 Y = 1000 1000 
Summary of pr.k: 
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0100  0.2280  0.3980  0.3724  0.5306  0.6340 
SE(initial) by SRS, Pois = 0.001989985 0.001989975 
SE(ACS_1) = 0.002402847 	 leading term: 0.002402847 
SE(ACS_2) = 0.002517724 
SE(ACS_panel) = 0.001695502 	 RE(analytic) = 0.7259412 
               Panel          pACS          iACS
MC-Mean -0.000100000 -0.0003612731 -9.474408e-05
MC-SD    0.002082483  0.0018322155  1.558296e-03
RE(pACS, iACS) = 0.7740879 0.5599339 


In [17]:
mainEpiPanel(M1=10,M12=5,Y12=100,emerge=T,f=0.01,lift=1,B=50,go=T) # Large, slowly emerging

(M, max(K(t,t+1))) = 15 100 90 	 Y = 1000 1000 
Summary of pr.k: 
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0100  0.2280  0.3980  0.3724  0.5306  0.6340 
SE(initial) by SRS, Pois = 0.001407132 0.001407125 
SE(ACS_1) = 0.002402847 	 leading term: 0.002402847 
SE(ACS_2) = 0.00253094 
SE(ACS_panel) = 0.001336681 	 RE(analytic) = 0.9023813 
               Panel          pACS         iACS
MC-Mean -0.000180000 -6.281966e-05 3.054481e-05
MC-SD    0.001547744  1.233772e-03 1.099662e-03
RE(pACS, iACS) = 0.6354361 0.5048014 


In [18]:
mainEpiPanel(M1=500,M12=10,Y12=400,emerge=F,f=0.01,lift=1,B=50,go=F) # Small, quickly evolving

(M, max(K(t,t+1))) = 500 2 42 	 Y = 1000 1000 
Summary of pr.k: 
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0100  0.1069  0.1943  0.1883  0.2732  0.3443 
SE(initial) by SRS, Pois = 0.002814264 0.002814249 
SE(ACS_1) = 0.003138511 	 leading term: 0.003138511 
SE(ACS_2) = 0.003018708 
              Panel         pACS         iACS
MC-Mean 0.000460000 0.0003176524 0.0004494472
MC-SD   0.002815482 0.0024203680 0.0026275474
RE(pACS, iACS) = 0.7390219 0.8709548 


In [19]:
mainEpiPanel(M1=500,M12=10,Y12=400,emerge=T,f=0.01,lift=1,B=50,go=F) # Small, quickly emerging

(M, max(K(t,t+1))) = 510 2 40 	 Y = 1000 1000 
Summary of pr.k: 
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0100  0.1024  0.1862  0.1807  0.2621  0.3310 
SE(initial) by SRS, Pois = 0.002814264 0.002814249 
SE(ACS_1) = 0.003138511 	 leading term: 0.003138511 
SE(ACS_2) = 0.003025693 
              Panel         pACS        iACS
MC-Mean 0.000280000 0.0001448542 0.000160000
MC-SD   0.002507458 0.0023523558 0.002413675
RE(pACS, iACS) = 0.8801133 0.9265956 


In [20]:
mainEpiPanel(M1=500,M12=50,Y12=100,emerge=T,f=0.01,lift=1,B=50,go=F) # Small, slowly emerging

(M, max(K(t,t+1))) = 550 2 2 	 Y = 1000 1000 
Summary of pr.k: 
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.01000 0.01247 0.01495 0.01495 0.01742 0.01990 
SE(initial) by SRS, Pois = 0.001407132 0.001407125 
SE(ACS_1) = 0.003138511 	 leading term: 0.003138511 
SE(ACS_2) = 0.003139272 
               Panel          pACS          iACS
MC-Mean -0.000400000 -0.0003883417 -0.0004465327
MC-SD    0.001340119  0.0013456178  0.0012552078
RE(pACS, iACS) = 1.008224 0.877293 
